¿Es posible realizar predicciones sobre el destino de un cliente utilizando variables predictoras como el tiempo, el lugar de partida, la edad y el identificador del cliente?

En esta notebook exploramos distintos clasificadores asi como variaciones de los mismos para ver el proder predictivo que se tiene para identificar donde un usuario terminara basado en la informacion que tenemos al inicio de su viaje.

# Librerias

In [1]:
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

c:\Users\joses\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Dataset

Vamos a preparar nuestros datos para poder ser usados por los distintos clasificadores que nos proporciona Sklearn.

In [2]:
BiciDF = pd.read_csv("../data/datos_abiertos_2023_03.csv", encoding = 'latin-1')

# Parse String to Datatime
BiciDF["Inicio_del_viaje"] = pd.to_datetime(BiciDF["Inicio_del_viaje"],
               format='%Y-%m-%d %H:%M:%S')
BiciDF["Fin_del_viaje"] = pd.to_datetime(BiciDF["Fin_del_viaje"],
               format='%Y-%m-%d %H:%M:%S')

# Make the year a int instead of float
BiciDF = BiciDF.dropna()
BiciDF["Anio_de_nacimiento"] = BiciDF["Anio_de_nacimiento"].astype(int)

BiciDF.head()

,Viaje_Id,Usuario_Id,Genero,Anio_de_nacimiento,Inicio_del_viaje,Fin_del_viaje,Origen_Id,Destino_Id
0,27598660,12551,M,1987,2023-03-01 00:00:32,2023-03-01 00:11:09,80,39
1,27598661,1153034,F,1994,2023-03-01 00:00:38,2023-03-01 00:09:47,188,17
2,27598668,1566425,M,1973,2023-03-01 00:04:52,2023-03-01 00:20:43,86,4
3,27598678,556440,M,2000,2023-03-01 00:05:55,2023-03-01 00:18:36,85,85
4,27598686,344644,M,2000,2023-03-01 00:07:33,2023-03-01 00:18:54,241,198


Tenemos que hacer un parsing de los datos que no son enteros en este caso las caracteristicas que vamos a manejar que no son enteros son el Genero y el inicio del viaje. El genero es sencillo como designa un genero como 1 y al otros con el 0. Por otro lado, para el inicio de viaje una opcion seria pasar todo a UNIX time; sin embargo, esto no da mucha informacion a un clasificador y es una codificacion bastante burda.

Por la naturaleza de los datos uno puede intuir que se tienen ciertos patrones semanales e interesa mas analizar estos patrones para el clasificador que solo el UNIX time. Es decir, es posible que los tiempos de los lunes se parezcan mas que a los de otro lunes que a los del miercoles de la misma semana. Entonces, vamos tomar el *tiempos que ha pasado desde el inicio de semana*. Ademas de eso para tener una mejor agrupacion tomaremos intervalos de 15 minutos. Es decir, todos los Lunes de $0:00$ a $0:15$ seran identificados con la etiqueta $0$, los del $0:15$ asl $0:30$ con la etiqueta $1$ y asi sucesivamente. La funcion mostrada a continuacion hace justamente ese mapping de los tiempos.

In [3]:
def parse_datetime(dt):
    start_of_week = dt - datetime.timedelta(days=dt.weekday())
    start_of_week = start_of_week.replace(hour=0, minute=0, second=0)
    elapsed_time = dt - start_of_week
    minutes = elapsed_time.total_seconds()/60
    return int(minutes) // 15

parse_datetime(datetime.datetime.now())

533

En la siguiente celda dividimos nuestros datos en un conjunto de entrenamiento y uno de prueba para medir el rendimiento de los clasificadores. De igual forma hacemos los castings pertinentes y trabajamos con numpy arrays para mantener uniformidad en como trabajamos con las entradas y salidas de nuestros clasificadores.

In [6]:
def parseDF(df):
    # Make Gender a number
    df["Genero"] = df["Genero"].map(lambda x: x=='M')
    # Datetime to int
    df["Inicio_del_viaje"] = df["Inicio_del_viaje"].map(parse_datetime)
    return df

trainDF, testDF = train_test_split(BiciDF, test_size=0.2)
# Prepare train data
trainDF_features = trainDF[["Genero", "Anio_de_nacimiento", "Inicio_del_viaje", "Origen_Id"]]
train_features = parseDF(trainDF_features).to_numpy()
train_labels   = trainDF[["Destino_Id"]].to_numpy().flatten()

# Prepare test data
testDF_features = testDF[["Genero", "Anio_de_nacimiento", "Inicio_del_viaje", "Origen_Id"]]
test_features = parseDF(testDF_features).to_numpy()
test_labels   = trainDF[["Destino_Id"]].to_numpy().flatten()

C:\Users\joses\AppData\Local\Temp/ipykernel_16788/3558726538.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Genero"] = df["Genero"].map(lambda x: x=='M')
C:\Users\joses\AppData\Local\Temp/ipykernel_16788/3558726538.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Inicio_del_viaje"] = df["Inicio_del_viaje"].map(parse_datetime)


# Red Neuronal

In [7]:
NN = MLPClassifier(
    hidden_layer_sizes=(16, 32, 64), 
    random_state=1234, 
    verbose=True, # Print classifier progress
    max_iter=10 # Set max iteration to get results sooner
).fit(train_features, train_labels)

print(f"Training score: {NN.score(train_features, train_labels)}")
print(f"Test score: {NN.score(train_features, train_labels)}")

Iteration 1, loss = 6.76269608
Iteration 2, loss = 5.36395059
Iteration 3, loss = 5.28367663
Iteration 4, loss = 5.26012639
Iteration 5, loss = 5.24926198
Iteration 6, loss = 5.23652005
Iteration 7, loss = 5.21774525
Iteration 8, loss = 5.20762897
Iteration 9, loss = 5.19490405
Iteration 10, loss = 5.17934703


c:\Users\joses\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Training score: 0.033304324960556964
Test score: 0.033304324960556964
